In [46]:
import pandas as pd
import numpy as np

In [47]:
df1 = pd.read_csv('../datasets/tong_hop1.csv')
df2 = pd.read_csv('../datasets/tong_hop.csv')
df2 = df2[~df2.label.isna()]

df2.set_index('PID', inplace=True)
df1.set_index('PID', inplace=True)

df1.index = df1.index.astype(str)
df2.index = df2.index.astype(str)

df2 = df2.drop(columns=['indate_x', 'indate_y', 'HBH', 'HBB']).assign(sheet='d4')
df1.shape, df2.shape

((23121, 18), (11255, 20))

In [48]:
intersec_pid = set(df1.index) & set(df2.index)

In [49]:
df3 = df1[~df1.index.isin(intersec_pid)]
df3 = df3.assign(inyear=2017, label=((df3.alpha + df3.beta) > 0).astype(int))
df3 = df3.drop(columns=['alpha', 'beta'])

In [50]:
df = pd.concat([df3, df2])
# df.to_csv('../datasets/tong_hop2.csv')


In [51]:
# df.reset_index().PID.to_csv("../datasets/pid.csv")

In [52]:
address_df = pd.read_csv('../datasets/address2.csv')
address_df.drop_duplicates(subset='PID', keep='first', inplace=True)
address_df.set_index("PID", inplace=True)
address_df.index = address_df.index.astype(str)
address_df.shape

(26462, 2)

In [61]:
numerical_columns = ['SLHC', 'HST', 'HCT', 'MCV', 'MCH', 'MCHC', 'RDWCV', 'SLTC', 'SLBC', 'FE', 'FERRITIN', 'HBA1', 'HBA2', 'label']

final_df = df.merge(address_df, how='left', left_index=True, right_index=True)
final_df = final_df[final_df.sex.isin(['M', 'F']) & ~final_df.Age.isna()]
final_df.shape
full_df = final_df[numerical_columns].apply(pd.to_numeric,  errors='coerce', downcast='float').dropna()
# full_df = full_df[~((full_df.MCV.astype(float)>=85) & (full_df.MCH.astype(float)>=28))]
full_df2 = final_df[numerical_columns].apply(pd.to_numeric,  errors='coerce', downcast='float').dropna()
full_df = full_df[~full_df.index.isin(full_df[full_df.index.duplicated()].index)]

full_df.label.sum(), full_df.shape[0] - full_df.label.sum()
full_df = full_df.sample(frac=1)

In [62]:
# negative_df = final_df[(final_df.label==1) & ~final_df.FE.isna()& ~final_df.FERRITIN.isna()]
# negative_df = final_df[(final_df.label==0) & (~(final_df.HBA1.isna() | final_df.HBA2.isna() |final_df.FE.isna() | final_df.FERRITIN.isna()))]
negative_df = full_df[(full_df.label==0)]
positive_df = full_df[(full_df.label==1)]
# full_df.index.duplicated()

In [63]:
p_df = positive_df[~((positive_df.MCV.astype(float)>=85) & (positive_df.MCH.astype(float)>=28))].iloc[-1000:]
t_1 = negative_df[((negative_df.MCV.astype(float)>=85) & (negative_df.MCH.astype(float)>=28))].iloc[-635:]
t_2 = negative_df[~((negative_df.MCV.astype(float)>=85) & (negative_df.MCH.astype(float)>=28))].iloc[-365:]
# negative_df.shape
negative_df[((negative_df.MCV.astype(float)>=85) & (negative_df.MCH.astype(float)>=28))].shape

(635, 14)

In [64]:
nm_df = pd.concat([t_1, t_2])
nm_df

,SLHC,HST,HCT,MCV,MCH,MCHC,RDWCV,SLTC,SLBC,FE,FERRITIN,HBA1,HBA2,label
PID,,,,,,,,,,,,,,
1920064799,5.14,155.0,0.441,85.800003,30.100000,350.0,12.8,275.0,6.84,321.500000,20.200001,97.900002,2.1,0.0
1920020149,5.33,159.0,0.467,87.599998,29.799999,340.0,13.5,199.0,5.62,752.900024,15.200000,97.800003,2.2,0.0
1920047014,5.68,174.0,0.503,88.599998,30.700001,346.0,14.8,270.0,11.54,829.200012,11.200000,98.000000,2.0,0.0
14021154,4.12,125.0,0.377,91.500000,30.500000,333.0,12.7,185.0,5.10,14.300000,512.200012,95.500000,2.1,0.0
15026655,4.68,135.0,0.413,88.199997,28.900000,328.0,13.2,236.0,8.88,14.500000,240.199997,78.500000,1.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18034345,3.01,82.0,0.254,84.500000,27.299999,323.0,15.2,230.0,6.04,22.000000,9.000000,73.599998,1.0,0.0
1920031101,4.60,111.0,0.354,77.000000,24.200001,314.0,14.2,244.0,9.29,49.200001,17.400000,97.500000,2.5,0.0
97590014,4.70,125.0,0.392,83.300003,26.600000,319.0,15.9,234.0,10.20,10.700000,7.900000,98.000000,2.0,0.0


In [65]:
test_df = pd.concat([nm_df, p_df])
test_df = test_df.assign(train=0)
test_df.shape
test_address_df = final_df[final_df.index.isin(test_df.index)][['Age', 'sex']]
test_address_df  = test_address_df.reset_index().drop_duplicates(subset=['PID']).set_index('PID')

test_df = test_df.merge(test_address_df, how='left', left_index=True, right_index=True)


In [66]:
full_df2[~full_df2.index.isin(test_df.index)].shape, full_df2.shape

((8435, 14), (10435, 14))

In [67]:
train_df = final_df[~final_df.index.isin(test_df.index)]
train_df = train_df.assign(train=1)
train_df = train_df[
    (~((train_df.MCV.astype(float)>=85) & (train_df.MCH.astype(float)>=28)) & (train_df.label==1)) | (train_df.label==0)
]


In [68]:



splited_df = pd.concat([train_df, test_df])
splited_df.assign(thalas=splited_df.label).to_csv('../datasets/splited_train_test.csv')